# Alternating Leaset Squares (ALS)
- 최소제곱, Matrix Factorization의 변형
- Implicit Feedback 데이터 바탕으로 진행
- 연산속도가 빠르다 (병렬처리 가능)
- https://sungkee-book.tistory.com/13

In [1]:
%conda install -y scikit-learn pandas datasets tqdm
%pip install -U matplotlib

Channels:
 - defaults
Platform: linux-64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - implicit

Current channels:

  - https://repo.anaconda.com/pkgs/main
  - https://repo.anaconda.com/pkgs/r

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 6.3 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import implicit

In [3]:
# Data Load
data = load_dataset("nbtpj/movielens-1m-ratings")["train"].shuffle(seed=10).select(range(200000))
movielens_df = pd.DataFrame(data)
moveslens_df = movielens_df[["user_id", "movie_id", "user_rating"]]

# map 함수를 사용하여 원래의 사용자 id를 정수 인덱스로 변환
user_ids = movielens_df["user_id"].unique()
user_id_map = {id: index for index, id in enumerate(user_ids)}
movie_ids = movielens_df["movie_id"].unique()
movie_id_map = {id: index for index, id in enumerate(movie_ids)}

movielens_df["user_id"] = movielens_df["user_id"].map(user_id_map)
movielens_df["movie_id"] = movielens_df["movie_id"].map(movie_id_map)

In [4]:
# Decode
train_data, test_data = train_test_split(movielens_df, test_size=0.2, random_state=10)
ratings_matrix = csr_matrix(
    (train_data["user_rating"].astype(float), (train_data["user_id"], train_data["movie_id"])),
    shape=(len(user_ids), len(movie_ids))
)

In [5]:
model_als = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.05, iterations=20)
model_als.fit(ratings_matrix)

/usr/local/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
100%|██████████| 20/20 [00:00<00:00, 25.33it/s]


In [7]:
user_id = 0
k = 10

recommended = model_als.recommend(user_id, ratings_matrix[user_id], N=k)
df_recommended = pd.DataFrame(zip(*recommended), columns=["item_id", "score"])

df_recommended

,item_id,score
0,106,1.438559
1,569,1.069911
2,367,1.066749
3,1124,1.043647
4,374,1.039047
5,241,1.015718
6,116,1.004639
7,92,0.944094
8,38,0.939263
9,467,0.936985
